# Week1-Homework by Cynthia
## 實作推薦系統paart1

## 基礎建設

In [1]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [2]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2021-12-25 19:08:43--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv’

All_Beauty.csv      100%[===================>]  14.78M  23.3MB/s    in 0.6s    

2021-12-25 19:08:44 (23.3 MB/s) - ‘All_Beauty.csv’ saved [15499476/15499476]

--2021-12-25 19:08:44--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz’

meta_All_Beauty.jso 100%[===================>]   9.85M  18.5MB/s    in 0.5s    

202

In [3]:
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [4]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

In [6]:
# 將時間戳記欄位（'unixReviewTime'）轉換成比較容易理解的時間格式

ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

## 資料切分

In [7]:
# 切分資料
# training-2019/9/1以前
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]

# testing-2018/9/1~2018/9/30
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]

ratings_trainings

,asin,reviewerID,overall,unixReviewTime,DATE
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000,2015-02-19
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800,2014-12-18
2,0143026860,A1572GUYS7DGSR,4.0,1407628800,2014-08-10
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000,2013-03-11
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200,2011-12-25
...,...,...,...,...,...
371340,B01HJEGTYK,A202DCI7TV1022,1.0,1500508800,2017-07-20
371341,B01HJEGTYK,A3FSOR5IJOFIBE,5.0,1489622400,2017-03-16
371342,B01HJEGTYK,A1B5DK6CTP2P24,5.0,1488326400,2017-03-01
371343,B01HJEGTYK,A23OUYS5IRMJS9,2.0,1487635200,2017-02-21


In [8]:
# 整理資料表-方便做商品銷售數量計算

ratings_trainings['num'] = 1
ratings_trainings.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,asin,reviewerID,overall,unixReviewTime,DATE,num
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000,2015-02-19,1
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800,2014-12-18,1
2,0143026860,A1572GUYS7DGSR,4.0,1407628800,2014-08-10,1
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000,2013-03-11,1
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200,2011-12-25,1


In [10]:
# 將商品評分取中位數
# 品銷售日期取最近一次銷售
# 計算商品被購買的次數

ratings_trainings2 = ratings_trainings.groupby('asin', as_index=False).agg({'overall':'median', 'DATE':'max', 'num':'count'})
ratings_trainings2.head()

,asin,overall,DATE,num
0,0061073717,5.0,2000-06-03,2
1,0143026860,5.0,2017-10-23,17
2,014789302X,5.0,2018-04-06,20
3,0571348351,5.0,2018-06-23,3
4,0692508988,5.0,2016-03-24,1


In [11]:
# 將整理後的資料表做排序，方便觀察資料集狀態

ratings_trainings3 = ratings_trainings2.sort_values(by=['num', 'overall', 'DATE'], ascending=[False, False, False])
ratings_trainings3.head()

,asin,overall,DATE,num
939,B000FOI48G,5.0,2018-07-07,8672
1059,B000GLRREU,5.0,2018-05-21,8341
59,1620213982,5.0,2018-05-14,4792
3301,B001QY8QXM,5.0,2018-06-08,4544
29552,B01DKQAXC0,5.0,2018-08-31,4197


In [12]:
# 接續整理users資料表

ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

## 產生推薦

In [13]:
# 此次推薦採有條件的隨機抽樣
# 先設定樣本條件
# 商品銷售數量大於2000筆(取較熱門的商品)
# 日期晚於2015-01-01(取較近期銷售)
#　評分中位數大於４分（評價較好之商品）

condition = ratings_trainings3[(ratings_trainings3.num > 2000) & (ratings_trainings3.DATE > '2015-01-01')\
                               & (ratings_trainings3.overall > 4.0)]
condition.head()                               

,asin,overall,DATE,num
939,B000FOI48G,5.0,2018-07-07,8672
1059,B000GLRREU,5.0,2018-05-21,8341
59,1620213982,5.0,2018-05-14,4792
3301,B001QY8QXM,5.0,2018-06-08,4544
29552,B01DKQAXC0,5.0,2018-08-31,4197


In [17]:
# 推薦－有條件隨機抽樣、且取後放回
def recommender(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    '''
    random-based (Your Code)-實現推薦的地方
    '''
    
    condition = training_data
    recommendations = {user: condition['asin'].sample(n=k, replace=True).tolist() for user in users}
    
    return recommendations
ratings_by_user = recommender(condition, users)
ratings_by_user

{'A100XQFWKQ30O2': ['B000GLRREU',
  'B000WYJTZG',
  'B000WYJTZG',
  'B00W259T7G',
  'B0012Y0ZG2',
  'B0012Y0ZG2',
  'B01DKQAXC0',
  'B000FOI48G',
  'B00W259T7G',
  'B0012Y0ZG2'],
 'A103T1QOGFCSEH': ['B00W259T7G',
  'B000FOI48G',
  'B000GLRREU',
  'B00W259T7G',
  'B01DKQAXC0',
  'B000FOI48G',
  'B00W259T7G',
  'B00W259T7G',
  'B006IB5T4W',
  'B001QY8QXM'],
 'A106UKKSJ2KXPF': ['B00W259T7G',
  'B000GLRREU',
  'B01DKQAXC0',
  'B001QY8QXM',
  'B0012Y0ZG2',
  'B00005JS5C',
  '1620213982',
  'B00W259T7G',
  'B001QY8QXM',
  'B01DKQAXC0'],
 'A10A7GV4D5A11V': ['1620213982',
  'B00005JS5C',
  'B000GLRREU',
  'B00W259T7G',
  'B001QY8QXM',
  '1620213982',
  'B0012Y0ZG2',
  'B00W259T7G',
  'B01DKQAXC0',
  'B00W259T7G'],
 'A1119JJ37ZLB8R': ['B006IB5T4W',
  '1620213982',
  'B000GLRREU',
  'B00W259T7G',
  'B01DKQAXC0',
  'B001QY8QXM',
  'B0012Y0ZG2',
  'B01DKQAXC0',
  'B00W259T7G',
  'B01DKQAXC0'],
 'A113UOOLBSZN52': ['B01DKQAXC0',
  'B00W259T7G',
  'B001QY8QXM',
  'B00005JS5C',
  'B006IB5T4W',
  '1620

## 結果評估

In [18]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

evaluate(ratings_testings_by_user, ratings_by_user)

0.05423728813559322